### Homework 2

Chris Dailey

christopher.dailey@gmail.com

W261-4

## Initial HDFS setup and testing

In [1]:
%%writefile wordcount.txt
hello hi hi hallo
bonjour hola hi ciao
nihao konnichiwa ola
hola nihao hello

Writing wordcount.txt


### Mapper

In [2]:
%%writefile mapper.py
#!/usr/bin/python
import sys
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split the line into words
    words = line.split()
    # increase counters
    for word in words:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        #
        # tab-delimited; the trivial word count is 1
        print(word+", 1")

Writing mapper.py


In [34]:
!chmod +x mapper.py

In [4]:
!cat wordcount.txt | python mapper.py

hello, 1
hi, 1
hi, 1
hallo, 1
bonjour, 1
hola, 1
hi, 1
ciao, 1
nihao, 1
konnichiwa, 1
ola, 1
hola, 1
nihao, 1
hello, 1


### Reducer

In [15]:
%%writefile reducer.py
#!/usr/bin/python
from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count = line.split(",")

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print(current_word + "\t" + str(current_count))
        current_count = count
        current_word = word

# do not forget to output the last word if needed!
if current_word == word:
    print(current_word + "\t" + str(current_count))

Overwriting reducer.py


In [35]:
!chmod +x reducer.py

In [17]:
!echo "foo foo quux labs foo bar quux" | python mapper.py | sort -k1,1 | python reducer.py

bar	1
foo	3
labs	1
quux	2


In [32]:
!hadoop/bin/hdfs dfs -rm /user/ubuntu/wordcountOutput/*
!hadoop/bin/hdfs dfs -rmdir /user/ubuntu/wordcountOutput

rm: `/user/ubuntu/wordcountOutput/*': No such file or directory


In [19]:
hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar -file ~/repos/261/HW2/mapper.py -mapper ~/repos/261/HW2/mapper.py -file ~/repos/261/HW2/reducer.py -reducer ~/repos/261/HW2/reducer.py -input /user/ubuntu/wordcount.txt -output /user/ubuntu/wordcountOutput

/bin/sh: 1: hdfs: not found


In [27]:
!hadoop/bin/hdfs dfs -cat /user/ubuntu/wordcountOutput/part-00000

bonjour	1
ciao	1
hallo	1
hello	2
hi	3
hola	2
konnichiwa	1
nihao	2
ola	1


In [36]:
!hadoop/bin/hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar -mapper ~/repos/261/HW2/mapper.py -reducer ~/repos/261/HW2/reducer.py -input /user/ubuntu/wordcount.txt -output /user/ubuntu/wordcountOutput

packageJobJar: [/tmp/hadoop-unjar3869171142529360995/] [] /tmp/streamjob4695714299996201914.jar tmpDir=null
16/01/24 18:10:18 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/24 18:10:19 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/24 18:10:19 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/24 18:10:19 INFO mapreduce.JobSubmitter: number of splits:2
16/01/24 18:10:19 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453569821305_0011
16/01/24 18:10:19 INFO impl.YarnClientImpl: Submitted application application_1453569821305_0011
16/01/24 18:10:19 INFO mapreduce.Job: The url to track the job: http://ip-172-31-54-132:8088/proxy/application_1453569821305_0011/
16/01/24 18:10:19 INFO mapreduce.Job: Running job: job_1453569821305_0011
16/01/24 18:10:24 INFO mapreduce.Job: Job job_1453569821305_0011 running in uber mode : false
16/01/24 18:10:24 INFO mapreduce.Job:  map 0% reduce 0%
16/01/24 18:10:31 INFO ma

In [37]:
!hadoop/bin/hdfs dfs -cat /user/ubuntu/wordcountOutput/part-00000

bonjour	1
ciao	1
hallo	1
hello	2
hi	3
hola	2
konnichiwa	1
nihao	2
ola	1


## HW 2.1
Sort in Hadoop MapReduce

In [38]:
!hadoop/bin/hdfs dfs -mkdir /user/ubuntu/sort

In [54]:
import random as rnd

integers = open("integers.txt", 'w')

for i in range(10000):
    integers.write(str(int(rnd.random()*5000))+", \n")
integers.close()
print("done")

done


In [55]:
%%writefile mapper.py
#!/usr/bin/python
import sys
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split the line into words
    integer, blank = line.split(",")
    print(str(integer))

Overwriting mapper.py


In [56]:
!chmod +x mapper.py

In [57]:
!cat integers_mini.txt | python mapper.py

4769
1384
1146
4746
466
3275
1069
4269
393
4175
2866
1955
3260
3609
3705
741
2611
1358
1646
3133
2939
963
2826
4211
1028
3454
2789
4266
3610
3478
4641
3393
4452
4939
3638
4008
260
2378
4829
3812
3976
1166
3179
418
4262
1117
3324
3655
4767
2066


In [59]:
%%writefile reducer.py
#!/usr/bin/python
import sys
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    print(line)

Overwriting reducer.py


In [60]:
!chmod +x reducer.py

In [62]:
!hadoop/bin/hadoop jar /home/ubuntu/hadoop-2.7.1/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
#-D stream.num.map.output.key.fields=1 \
#-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
#-D mapred.text.key.comparator.options=-n \
-mapper /home/ubuntu/repos/261/HW2/mapper.py \
-reducer /home/ubuntu/repos/261/HW2/reducer.py \
-input /user/ubuntu/sort/integers.txt \
-output /user/ubuntu/sort/output

packageJobJar: [/tmp/hadoop-unjar7335385830326096165/] [] /tmp/streamjob5204162095946303316.jar tmpDir=null
16/01/24 20:28:21 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/24 20:28:21 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/24 20:28:21 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/24 20:28:21 INFO mapreduce.JobSubmitter: number of splits:2
16/01/24 20:28:21 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
16/01/24 20:28:21 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1453569821305_0013
16/01/24 20:28:22 INFO impl.YarnClientImpl: Submitted application application_1453569821305_0013
16/01/24 20:28:22 INFO mapreduce.Job: The url to track the job: http://ip-172-31-54-132:8088/proxy/application_1453569821305_0013/
16/01/24 20:28:22 INFO mapreduce.Job: Running job: job_1453569821305_0013
16/01/24 20:28:27 IN

In [63]:
!hadoop/bin/hdfs dfs -ls /user/ubuntu/sort/output

Found 2 items
-rw-r--r--   1 ubuntu supergroup          0 2016-01-24 20:28 /user/ubuntu/sort/output/_SUCCESS
-rw-r--r--   1 ubuntu supergroup      77744 2016-01-24 20:28 /user/ubuntu/sort/output/part-00000


In [64]:
!hadoop/bin/hdfs dfs -cat /user/ubuntu/sort/output/part-00000

0	
	
1	
	
1	
	
2	
	
2	
	
3	
	
3	
	
4	
	
4	
	
4	
	
4	
	
5	
	
5	
	
6	
	
6	
	
6	
	
6	
	
7	
	
9	
	
9	
	
11	
	
12	
	
12	
	
12	
	
12	
	
12	
	
13	
	
13	
	
13	
	
14	
	
14	
	
16	
	
16	
	
16	
	
17	
	
17	
	
18	
	
19	
	
21	
	
21	
	
21	
	
23	
	
23	
	
23	
	
24	
	
24	
	
25	
	
26	
	
26	
	
28	
	
29	
	
29	
	
30	
	
30	
	
31	
	
31	
	
33	
	
33	
	
33	
	
34	
	
34	
	
34	
	
35	
	
35	
	
35	
	
38	
	
38	
	
39	
	
40	
	
41	
	
42	
	
42	
	
42	
	
43	
	
43	
	
43	
	
44	
	
44	
	
45	
	
46	
	
47	
	
48	
	
49	
	
49	
	
49	
	
49	
	
50	
	
50	
	
51	
	
52	
	
52	
	
53	
	
53	
	
53	
	
53	
	
54	
	
54	
	
55	
	
55	
	
56	
	
56	
	
56	
	
57	
	
58	
	
58	
	
58	
	
58	
	
59	
	
59	
	
59	
	
59	
	
59	
	
61	
	
61	
	
63	
	
63	
	
63	
	
64	
	
64	
	
65	
	
65	
	
66	
	
67	
	
67	
	
67	
	
67	
	
68	
	
69	
	
69	
	
70	
	
70	
	
71	
	
71	
	
72	
	
72	
	
73	
	
74	
	
74	
	
74	
	
77	
	
77	
	
77	
	
77	
	
78	
	
78	
	
78	
	
78	
	
79	
	
79	
	
80	
	
81	
	
81	
	
81	
	
82	
	
83	
	
83	
	
84	
	
85	
	
86	
	
87	
	
87	
	
88	
	
89	
	
90	
	
90	
	
90	
	
90	
	
91	
	
91	
	
92	
	


In [65]:
!hadoop/bin/hdfs dfs -rm /user/ubuntu/sort/output/*
!hadoop/bin/hdfs dfs -rmdir /user/ubuntu/sort/output

16/01/24 20:30:28 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/sort/output/_SUCCESS
16/01/24 20:30:28 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/sort/output/part-00000
